# Processing steps for galaxy clusters ARF analysis

In [63]:
import os
import subprocess
import sys
import glob

import numpy as np

from astropy.io import fits

In [54]:
#
# utility function
#
def run_command(command,verbose=True):
    #
    # Execute a shell command
    #
    try:
        result = subprocess.run(command, shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
        retcode=result.returncode
        if retcode < 0:
            if (verbose):
                print(f"Execution of {command} was terminated by signal {-retcode}", file=sys.stderr)
        else:
            if (verbose):
                print(f"Execution of {command} returned {retcode}", file=sys.stderr)
    except OSError as e:
        print(f"Execution of {command} failed:", e, file=sys.stderr)
    return retcode

## Setting up XMM-SAS

For some reason, running the ipython magic command `!. /home/xcaldata/setsas_161.sh`, does not propagate to the session, i.e. although it prints out that SAS is set up, it is not available after the execution of the shell command. So, I have to do it manually, by setting some environment variables.

The last line in the next cell, `!sasversion` should work.

In [49]:
#
# set up XMM-SAS
#
# the current one is 18.0 ==> /home/xcaldata/setsas_180.sh
# tests with XMM-SAS 16.1 for the arfgen
#!. /home/xcaldata/setsas_161.sh
#
#sas_dir = "/sas/Linux/RHEL_6.9Workstation/64/sas18_0_0"
sas_dir = "/sas/Linux/RHEL_6.9Workstation/64/sas16_1_0"
os.environ["SAS_DIR"]= sas_dir
os.environ["SAS_PATH"]=os.environ["SAS_DIR"]
os.environ["SAS_VERBOSITY"]="4"
os.environ["SAS_SUPPRESS_WARNING"]="1"
path = os.environ["PATH"]
os.environ["PATH"] = f"{sas_dir}/bin:{sas_dir}/binextra:{path}"
ld_path = os.environ["LD_LIBRARY_PATH"]
lib_path = f"{sas_dir}/lib:{sas_dir}/libextra:{sas_dir}/libsys:{ld_path}"
os.environ["LD_LIBRARY_PATH"] = lib_path
#
# check
#print ("PATH = ",os.environ["PATH"])
#print ("LF_LIBRARY_PATH = ",os.environ["LD_LIBRARY_PATH"])
#for ikey in os.environ.keys():
#    if ('SAS' in ikey):
#        print (ikey," = ",os.environ[ikey])
!sasversion

In [61]:
#
# check for consistency the SAS version
#
status = run_command('sasversion')
if (status != 0):
    print ('XMM-SAS is not available. Cannot continue.')
    raise Exception
else:
    print ("Found XMM-SAS.")
#

Found XMM-SAS.


Execution of sasversion returned 0


## Processing of a target

Next cell sets up the target name, the XMM `OBS_ID` and the different paths. I usually keep the following structure in the `root_dir`:
```
<target>
   <OBS_ID 1>
       <proc>
   <OBS_ID 2>
       <proc>
   ...
```
where in folders `<OBS_ID>` I keep the ODF files and the CCF file. While in `<proc>` I keep the processing products, like event lists, images etc. The spectral extraction regions are also kept in `<proc>` folder.

For processing with different versions of SAS or calibration, I use different names for the `<proc>` folders.

In [62]:
#
# set up the paths, the target and the OBS_ID
#
root_dir = "/lhome/ivaltchanov/XMM-clusters/"
target="A1795"
obsid="0097820101"
# the output processing will be saved to a folder proc_161
pps="proc_161"
odf_dir = f"{root_dir}/{target}/{obsid}"
#
if (not os.path.isdir(odf_dir)):
    print (f"{odf_dir} does not exist. Cannot continue.")
    raise FileNotFoundError
else:
    print (f"Found ODF folder: {odf_dir}.")
#
pps_dir = f"{root_dir}/{target}/{obsid}/{pps}"
if (not os.path.isdir(pps_dir)):
    print (f"{pps_dir} does not exist. Will create it.")
    os.mkdir(pps_dir)
else:
    print (f"Will use {pps_dir} for the products.")
#
os.environ['SAS_ODF'] = odf_dir
os.environ['SAS_CCFPATH'] = '/ccf/pub'

Found ODF folder: /lhome/ivaltchanov/XMM-clusters//A1795/0097820101.
Will use /lhome/ivaltchanov/XMM-clusters//A1795/0097820101/proc_161 for the products.


## Generating the current calibration file

Presumably the ODF files for the `target` and `OBS_ID` are already downloaded and avalable in the `odf_dir`.

Then we regenerate the current calibration files (CCF) and set the environment `SAS_CCF` to point to this file.

In [6]:
# **Step 1:** Assuming the ODF is already available in odf_dir, then run cifbuild
os.chdir(odf_dir)
#
# For tests with XMM-SAS v16.1, set the corretc analysis date
#
cif_file = f"{odf_dir}/ccf_161.cif"
comm = "cifbuild calindexset=ccf.cif analysisdate=\"2017-07-20T00:00:00\""
status = run_command(comm)
if (status != 0):
    raise Exception
#
os.environ['SAS_CCF'] = cif_file

## Generating lightcurves at high energy

Some observations may suffer from periods of high backgrounds. The lightcurves at high energy (10-12 keV) are a good representation of the overall background behaviour during an observation.

In [6]:
# **Step 2:** Genrerate the lightcurves
os.chdir(pps_dir)
#
evlists = glob.glob("*ImagingEvts.ds")
nev = len(evlists)
if (nev == 0):
    print(f"No ImagingEvts files found in folder {pps_dir}")
    raise FileNotFoundError
else:
    print (f"Found {nev} calibrated event lists")
#
for ev in evlists:
    iev = os.path.basename(ev)
     print (f"Generating lightcurve for {iev}")
    # the exposure
    expo = iev.split('_')[3]
    if ('EMOS1' in iev):
        rate = f'rate_mos1_{expo}.fits'
        expr = '#XMMEA_EM && (PI>10000) && (PATTERN==0)'
    elif ('EMOS2' in iev):
        rate = f'rate_mos2_{expo}.fits'
        expr = '#XMMEA_EM && (PI>10000) && (PATTERN==0)'
    elif ('EPN' in iev):
        rate = f'rate_pn_{expo}.fits'
        expr = ' #XMMEA_EP && (PI>10000&&PI<12000) && (PATTERN==0)'        
    #    
    command = f'evselect table={iev} withrateset=Y rateset={rate}' + \
    ' maketimecolumn=Y timebinsize=100 makeratecolumn=Y' + \
    f' expression=\'{expr}\''
    status = run_command(command)
    if (status != 0):
        print (f"Command \"{command}\" failed")
        raise Exception
#

## Filtering for good time intervals (GTI)

Based on the lightcurve count rates, we select the periods where the background is below a certain threshold in counts/s. The suggested default threshold for MOS detectors is 0.35 counts/s, while for pn it's 0.4 counts/s. If the median + 3 times the median absolute deviation of the observed rate is greater than the default values, then this is the new threshold for the filtering.

At the end, some diagnostics are printed out, e.g. the fraction of the good-time-interval.

In [ ]:
# **Step 3:** use the rate curves and select the GTI, semi-automatic
#
t = {}
time_min = 1.0e10
# get the min time, to be used for relative time
output = ""
for inst in ['mos1','mos2','pn']:
    #
    ratefiles = glob.glob(f"rate_{inst}*")
    for j in ratefiles:
        rate_hdu = fits.open(j)
        t = rate_hdu['RATE'].data
        ontime0 = rate_hdu['RATE'].header['EXPOSURE']
        #time_min = min(np.min(rate_hdu['RATE'].data['TIME']),time_min)
        # find the count-rate limit for filtering
        # median + 3*MAD
        #
        medrate = np.median(t["RATE"])
        xmad = mad_std(t['RATE'])
        ulimit = medrate + 3*xmad
        if ('mos' in inst):
            rate_lim = 0.35
        else:
            rate_lim = 0.4
        #
        use_limit = max(ulimit,rate_lim)
        print(f"{j}: actual rate limit to use for GTI filtering: {use_limit:.2f}")
        #
        gtiset = j.replace("rate","gti")
        gti_command = f'tabgtigen table={j} expression="RATE<={use_limit:.2f}" gtiset={gtiset}'
        #
        status = run_command(gti_command)
        if (status != 0):
            print (f"Command {gti_command} failed")
            raise Exception
        gti_hdu = fits.open(gtiset)
        ontime = gti_hdu[1].header['ONTIME']
        fraction = ontime/ontime0
        output += f"{j}: filtered GTI {ontime:.1f} from exposure {ontime0:.1f}, GTI fraction {fraction:.3f}\n"
#
print (output)

In [ ]:
# step 3a
# plot the lightcurve and the GTI per detector
#
# TBD

In [ ]:
# step 3b
# if needed, merge the three GTI
#
# TBD

## Create filtered event lists

The filtering of the event lists is using the standard flags and either the user provided GTI file or the ones generated during the previous step. The GTI files can be per detector or a merged one.

In [6]:
#**Step 4:** Create GTI filtered event lists, using the GTI file
#
# GTI file from Jukka Nevalainen
gtifile = f"{root_dir}/{target}/{target}_GTI.fits"
#
for jl in evlists:
    expo = jl.split('_')[3]
    if ('_EMOS1' in jl):
        if (gtifile == None):
            expr = f"#XMMEA_EM && gti(gti_mos1_{expo}.fits,TIME) && (PI>150)"
        else:
            expr = f"#XMMEA_EM && gti({gtifile},TIME) && (PI>150)"
        inst = 'mos1'
    elif ('_EMOS2' in jl):
        if (gtifile == None):
            expr = f"#XMMEA_EM && gti(gti_mos2_{expo}.fits,TIME) && (PI>150)"
        else:
            expr = f"#XMMEA_EM && gti({gtifile},TIME) && (PI>150)"
        inst = 'mos2'
    elif ('_EPN' in jl):
        if (gtifile == None):
            expr = f"#XMMEA_EP && gti(gti_pn_{expo}.fits,TIME) && (PI>150)"
        else:
            expr = f"#XMMEA_EP && gti({gtifile},TIME) && (PI>150)"
        inst = 'pn'
    else:
        raise Exception(f'Cannot identify the instrument in {jl}')
    #
    ev_command = f'evselect table={jl} withfilteredset=Y filteredset={inst}_evlist_clean.fits' +  \
        f' destruct=Y keepfilteroutput=T expression=\'{expr}\''
    #
    print (f"Filtering {inst} with GTI")
    status = run_command(ev_command)
    if (status != 0):
        print (f"Command \"{ev_command}\" failed.")
        raise Exception
#

## Generating images

Once we have the clean event lists we can use them to generate images in user-provided energy ranges. There is a large flexibility in the choices for the event selection (patterns, flags etc) and the output image properties, e.g. the pixel size. 

In [6]:
#**Step 5:** Generate images in user-selected energy range
pi0 = 500
pi1 = 7000
# image binning, 80 means 4" pixel
bin_size = 80
#
print (f"*** Generating images in band [{pi0},{pi1}] eV")
for inst in ['mos1','mos2','pn']:
    evlist = f'{inst}_evlist_clean.fits'
    if (not os.path.isfile(evlist)):
        print(f"Cannot find cleaned event lists for {inst}. Image will not be generated.")
        continue
    image_name = f'{inst}_image_{pi0}_{pi1}.fits'
    #
    if ('mos' in inst):
        expr = f'PI in [{pi0}:{pi1}] &&  #XMMEA_EM && PATTERN in [0:12]'
    else:
        expr = f'PI in [{pi0}:{pi1}] &&  FLAG==0 && PATTERN in [0:4]'
    #    
    ev_command = f'evselect table={evlist} xcolumn=X ycolumn=Y imagebinning=binSize' +  \
         f' ximagebinsize={bin_size} yimagebinsize={bin_size}' + \
         f' expression=\'{expr}\'' +  \
         f' withimageset=true imageset={image_name}'
    status = run_command(ev_command)
    if (status != 0):
        raise Exception

In [ ]:
# step 5a
# Display the images
#
# TBD

## Generate exposure maps, count-rate and detector mask images(optional)

This step is optional but it is needed in some cases, e.g. for combining MOS and pn images into a merged one you need the count-rate image.

In [ ]:
# step 5b
# generate the exposure image and the detector mask
#
do_detmask = True
do_crimage = True
print ("*** Generating exposure maps")
#
# need the attitude file
atthk = glob.glob("*_AttHk.ds")
if (not os.path.isfile(atthk[0])):
    print("Cannot find Attitude HK file, cannot continue.")
    raise FileNotFoundError
#
for inst in ['mos1','mos2','pn']:
    evlist = f'{inst}_evlist_clean.fits'
    image_name = f'{inst}_image_{pi0}_{pi1}.fits'
    if (not os.path.isfile(image_name)):
        print("Cannot find image {image_name}, cannot produce exposure image and detector mask.")
        continue
    #
    expfile = f"{inst}_expimage_{pi0}_{pi1}.fits"
    xcommand = f'eexpmap imageset={image_name} attitudeset={atthk[0]}' + \
    f' eventset={evlist} expimageset={expfile} pimin={pi0} pimax={pi1}'
    status = run_command(xcommand)
    if (status != 0):
        raise Exception
    #
    # now the detector map
    #
    if (do_detmask):
        detfile = f"{inst}_detmask_{pi0}_{pi1}.fits"
        xcommand = f'emask expimageset={expfile} detmaskset={detfile}' + \
        ' threshold1=0.3 threshold2=0.5'
        status = run_command(xcommand)
        if (status != 0):
            raise Exception
    #
    # now the count-rate image
    #
    if (do_crimage):
        crfile = f"{inst}_crimage_{pi0}_{pi1}.fits"
        hdu0 = fits.open(image_name)
        hdu1 = fits.open(expfile)
        # avoid division by zero 
        crdata = np.divide(hdu0[0].data,hdu1[0].data,where=(hdu1[0].data > 0.0))
        # the count-rate image will have the same header as the original image
        hdu0[0].data = cts
        hdu0.writeto(crfile)
        hdu1.close()
        hdu0.close()
#

## Instrument specific part, only processing EPIC-pn

### Source spectrum extraction


In [6]:
#**Step 6:** Generate a spectrum from a user provided region file. This also runs `backscale` on the source spectrum.
#
# this is per instrument and we only process PN at this stage.
#
inst = "pn"
regfile = f"{pps_dir}/{target}_{obsid}_src_region.reg"
with open(regfile) as reg:
    reg_line = reg.readline()
src_reg = reg_line.strip().split()[0]
#
print (src_reg)
#
evlist = f'{inst}_evlist_clean.fits'
spec_name = f'{target}_{obsid}_{inst}_spectrum_src.fits'
#
if ('pn' in inst):
    spec_chan_max = 20479
    expr1 = "(FLAG==0) && (PATTERN<=4)"
else:
    spec_chan_max = 11999
    expr1 = "#XMMEA_EM && (PATTERN<=12)"
#
ev_command = f"evselect table={evlist} withspectrumset=yes spectrumset={spec_name}" +  \
f" energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax={spec_chan_max}" +  \
f" expression='{expr1} && ((X,Y) IN {src_reg})'"
status = run_command(ev_command)
if (status != 0):
    raise Exception
#
# Now backscale the source spectrum
#
print ("*** Backscale the source spectrum")
xcommand = f"backscale spectrumset={spec_name} badpixlocation={evlist}"
status = run_command(xcommand)
if (status != 0):
    raise Exception

### Background spectrum extraction

In [ ]:
#**Step 6b:** Generate the background spectrum from a user provided region file. This also runs `backscale` on the source spectrum.
#
# this is per instrument and we only process PN at this stage.
#
inst = "pn"
regfile = f"{pps_dir}/{target}_{obsid}_bkg_region.reg"
with open(regfile) as reg:
    reg_line = reg.readline()
src_reg = reg_line.strip().split()[0]
#
print (src_reg)
#
evlist = f'{inst}_evlist_clean.fits'
spec_name = f'{target}_{obsid}_{inst}_spectrum_bkg.fits'
#
if ('pn' in inst):
    spec_chan_max = 20479
    expr1 = "(FLAG==0) && (PATTERN<=4)"
else:
    spec_chan_max = 11999
    expr1 = "#XMMEA_EM && (PATTERN<=12)"
#
ev_command = f"evselect table={evlist} withspectrumset=yes spectrumset={spec_name}" +  \
f" energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax={spec_chan_max}" +  \
f" expression='{expr1} && ((X,Y) IN {src_reg})'"
status = run_command(ev_command)
if (status != 0):
    raise Exception
#
# Now backscale the source spectrum
#
print ("*** Backscale the source spectrum")
xcommand = f"backscale spectrumset={spec_name} badpixlocation={evlist}"
status = run_command(xcommand)
if (status != 0):
    raise Exception

### Generate the RMF for the source spectrum

In [6]:
#**Step 7:** Generate an RMF file for the source spectrum
evlist = f'{inst}_evlist_clean.fits'
if (not os.path.isfile(evlist)):
    print("Cannot find cleaned event lists")
    raise FileNotFoundError
spec_name = f'{target}_{obsid}_{inst}_spectrum_src.fits'
if (not os.path.isfile(spec_name)):
    print (f"Cannot find source spectrum {spec_name}")
    raise FileNotFoundError
#
rmfset = f'{target}_{obsid}_{inst}_spectrum_src.rmf'
#
xcommand = f"rmfgen spectrumset={spec_name} rmfset={rmfset}"
print (f"*** Running {xcommand}")
status = run_command(xcommand)
if (status != 0):
    raise Exception

### Generate the ARF for the source extraction region

For extended sources, the parameters should be changed correspondingly.

In [6]:
#**Step 8:** Generate an RMF file for the source spectrum
# now generate the ARF
arfset = f'{target}_{obsid}_{inst}_spectrum_src.arf'
# flat detector map (not good for extended sources)
#xcommand = f"arfgen spectrumset={spec_name} arfset={arfset} withrmfset=yes rmfset={rmfset}" + \
#    f" badpixlocation={evlist} detmaptype=flat"
#
# MOS1 detector map (better for extended sources)
mos1file = "mos1_image_500_7000.fits"
# better to have the MOS1 count-rate image, so that the vignetting is not applied twice!
#
xcommand = f"arfgen spectrumset={spec_name} arfset={arfset} withrmfset=yes rmfset={rmfset}" + \
    f" badpixlocation={evlist} detmaptype=dataset detmaparray={mos1file} extendedsource=yes"
print (f"*** Running {xcommand}")
status = run_command(xcommand)
if (status != 0):
    raise Exception